- plot for tissuetype acc at X 
- cos+jacc and 3methods ---- genes on x axis
- subsample a few times for accuracy?

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
mpl.rc("figure", dpi=400)

In [ ]:
# Data preparation
import cellsaw
import cellsaw.io_utils
import notebookhelper

# loading data-> [[anndata]] 
filenames = notebookhelper.filenames44
path = '/home/ubuntu/data/scdata/'
datasets = [[cellsaw.io_utils.nuread(sample_size=1000, 
                            sampleseed = seed,
                            dir = path, 
                            dataset =f) for f in filenames]
                                for seed in [42,1337,31337,501,404]]

In [ ]:
# get genescores according to different pp methods 
# note how seurat is first so we can calculate values 
# before normalizing as normalize is 'auto'
preprocessingmethods = ['seurat_v3','meanexpression','cell_ranger','seurat', 'natto','meanexpressionnolog']
_ = [[cellsaw.preprocess.annotate_genescore(d, selector = pp) for d in datasets] for pp in preprocessingmethods]


In [ ]:
%%time
# lets collect data in a gigantic dataframe
# how did i do this before? 
import cellsaw.similarity.measures as measures
def mklist(rep_id,datasets,preprocessing,filenames):
    res = [ (rep_id,'cosine',genecount,preprocessing,
             *measures.adata_to_score(datasets,genecount, preprocessing, 'cosine', filenames)) 
                  for genecount in range(200,2000,50) ]
    res += [ (rep_id,'jaccard',genecount,preprocessing,
              *measures.adata_to_score(datasets,genecount, preprocessing, 'jaccard', filenames)) 
                  for genecount in range(200,4000,100) ]
    return res

result = [line for i,d in enumerate(datasets) 
               for pp in preprocessingmethods 
               for line in mklist(i,d,pp, filenames)]

## df.to_pickle("1_tissuesim_18B.pkl")
# df.to_pickle("1_tissuesim_21B.pkl") # contains mean no log

In [ ]:
# now we make a dataFrame
indexlabels = 'repeat similarity genecount preprocessing P@1 P@2 P@3'.split()

In [ ]:
import pandas as pd
df = pd.DataFrame(result,columns=indexlabels)
dff = df[df.similarity == 'jaccard']
# y axis 1.. .75 (maybe try meanexpression withouy log!)
sns.set_context("notebook")
sns.set_style('darkgrid')

niceprocessingmethods = ['Seurat v3','Mean expression','Cell Ranger','Seurat', 'Linear Seurat', 'Mean no log']
dff = dff.replace(preprocessingmethods, niceprocessingmethods)
sns.lineplot(data=dff, x='genecount', y='P@3', hue = 'preprocessing',ci='sd')
plt.title('Separating tissue types', fontsize = 24)
plt.xlabel('number of genes')
plt.ylabel('p@3±σ')


plt.show()
dff = df[df.similarity == 'cosine']
# y axis 1.. .75 (maybe try meanexpression withouy log!)
sns.set_context("notebook")
sns.set_style('darkgrid')

niceprocessingmethods = ['Seurat v3','Mean expression','Cell Ranger','Seurat', 'Linear Seurat', 'Mean no log']
dff = dff.replace(preprocessingmethods, niceprocessingmethods)
sns.lineplot(data=dff, x='genecount', y='P@3', hue = 'preprocessing',ci='sd')
plt.title('Separating tissue types', fontsize = 24)
plt.xlabel('number of genes')
plt.ylabel('p@3±σ')

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
dff = df[df.similarity == 'jaccard']
sns.lineplot(data=dff, x='genecount', y='P@1', hue = 'preprocessing')
plt.show()
sns.lineplot(data=dff, x='genecount', y='P@2', hue = 'preprocessing')
plt.show()
sns.lineplot(data=dff, x='genecount', y='P@3', hue = 'preprocessing')
plt.show()
dff = df[df.similarity == 'cosine']
sns.lineplot(data=dff, x='genecount', y='P@1', hue = 'preprocessing')
plt.show()
sns.lineplot(data=dff, x='genecount', y='P@2', hue = 'preprocessing')
plt.show()
sns.lineplot(data=dff, x='genecount', y='P@3', hue = 'preprocessing')
plt.show()